Our first task is to simulate the flow of heat in a solid.

The heat flow equation is:

$$ \frac{\partial u}{\partial t} = \frac{K}{C\rho}\nabla^2 u $$

$$ = \frac{K}{C\rho} \Big(\frac{\partial^2 u}{\partial x^2}
+ \frac{\partial^2 u}{\partial y^2}
+ \frac{\partial^2 u}{\partial z^2}\Big) $$

Where $K$ is the thermal conductivity, $C$ is the specific heat, and $\rho$ is the density.

We solve this using a timestep algorithm.

At a given point $u_i(t_{i})$, the spatial derivatives can be expressed in discrete form:
$$ \frac{\partial^2 u}{\partial x^2} = \frac{u_{i+1} - 2u_i + u_{i-1}}{\Delta x^2} $$

And the time derivative can be expressed as:
$$ \frac{\partial u}{\partial t} = \frac{u(t_{j+1}) - u(t_j)}{\Delta t} $$

Given a particular temperature distribution, we may find the next iteration using the following time step equation:

$$ u_{i,j,k}(t_{j+1}) = u_{i,j,k}(t_j)
 + \Big(
         \frac{u_{i+1,j,k} - 2u_{i,j,k} + u_{i-1,j,k}}{\Delta x^2}
         + \frac{u_{i,j+1,k} - 2u_{i,j,k} + u_{i,j-1,k}}{\Delta y^2}
         + \frac{u_{i,j,k+1} - 2u_{i,j,k} + u_{i,j,k-1}}{\Delta z^2}
    \Big) \Delta t $$

If we take the same spatial step size for each dimension, we may simplify the expression in parentheses:
$$ \Big(u_{i+1,j,k} + u_{i,j+1,k} + u_{i,j,k+1} + u_{i-1,j,k} + u_{i,j-1,k} + u_{i,j,k-1} - 6u_{i,j,k}\Big)
\cdot\frac{1}{\Delta x^2}
$$
$$
= \frac{1}{\Delta x^2} \Big(\sum_{adjacent} u(t_j)\Big) - 6u_{i,j,k}
$$

To handle an insulated boundary, we introduce fictional points such that the derivative will always be zero.

As a simplified case, we assume that the non-insulated boundaries are in perfect thermal contact with a heat sink at a constant temperature.
